In [1]:
!pip -q install --upgrade torch torchvision torchaudio
!pip -q install scikit-learn tqdm gradio opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 135.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

Imports, config, helpers

In [2]:
# Cell 2 — Imports, config, helpers (with stronger aug)
import os, json, random, time
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, random_split

from torchvision import transforms, datasets, models

import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

# Repro
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_SIZE = 224

def get_transforms(train=True):
    if train:
        return transforms.Compose([
            # Stronger, more realistic augmentation for street photos
            transforms.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0)),
            transforms.ColorJitter(0.3, 0.3, 0.3, 0.1),
            transforms.RandomApply([transforms.GaussianBlur(kernel_size=5)], p=0.3),
            transforms.RandomAffine(degrees=12, translate=(0.08, 0.08), scale=(0.9, 1.1)),
            transforms.RandomHorizontalFlip(p=0.2),
            transforms.ToTensor(),
            transforms.RandomErasing(p=0.25, scale=(0.02, 0.08), value='random'),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(IMG_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
        ])

def build_model(num_classes: int, pretrained=True):
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT if pretrained else None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def save_label_map(class_to_idx: dict, path="label_map.json"):
    idx_to_class = {int(v): k for k, v in class_to_idx.items()}
    with open(path, "w") as f:
        json.dump(idx_to_class, f, indent=2)

def load_label_map(path="label_map.json"):
    with open(path, "r") as f:
        d = json.load(f)
    return {int(k): v for k, v in d.items()}

def topk_probs(logits, k=5):
    probs = torch.softmax(logits, dim=-1)
    top_p, top_i = probs.topk(k)
    return top_p.detach().cpu().numpy().tolist(), top_i.detach().cpu().numpy().tolist()

print("Device:", DEVICE)

Device: cuda


Data

In [3]:
from torch.utils.data import DataLoader, random_split

DATA_ROOT = "./data"
VAL_SPLIT = 0.15
BATCH_SIZE = 64
NUM_WORKERS = 2

train_full = datasets.GTSRB(
    root=DATA_ROOT, split="train", download=True, transform=get_transforms(train=True)
)

try:
    # Some versions expose .classes / .class_to_idx
    class_names = list(train_full.classes)
    class_to_idx = dict(train_full.class_to_idx)
except Exception:
    # Fallback: derive from targets; make readable names like "class_0"... "class_42"
    class_ids = sorted({train_full[i][1] for i in range(len(train_full))})
    class_names = [f"class_{i}" for i in class_ids]
    class_to_idx = {name: idx for name, idx in zip(class_names, class_ids)}

num_classes = len(class_names)
print("Classes:", num_classes)

# Save label map for later inference/Gradio
save_label_map(class_to_idx, "label_map.json")

# Train/val split
n_total = len(train_full)
n_val = int(n_total * VAL_SPLIT)
n_train = n_total - n_val
train_set, val_set = random_split(
    train_full, [n_train, n_val], generator=torch.Generator().manual_seed(42)
)
# Use eval transforms for val set
val_set.dataset.transform = get_transforms(train=False)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=True)

len(train_set), len(val_set)

100%|██████████| 187M/187M [00:00<00:00, 223MB/s]


Classes: 43


(22644, 3996)

In [4]:
# Cell 4 — Train (AMP + cosine LR + label smoothing)
EPOCHS = 20               # a bit longer for stronger aug
LR = 3e-4
WEIGHT_DECAY = 1e-4
BEST_WEIGHTS = "model.pth"

model = build_model(num_classes=num_classes, pretrained=True).to(DEVICE)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

# Label smoothing to reduce overconfidence, help generalization
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

best_val_acc = 0.0

def accuracy(logits, y):
    return (logits.argmax(1) == y).float().mean().item()

for epoch in range(1, EPOCHS+1):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} [train]")
    run_loss = 0.0; run_acc = 0.0; n = 0

    for x, y in pbar:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        bs = y.size(0)
        run_loss += loss.item() * bs
        run_acc  += (logits.argmax(1) == y).float().sum().item()
        n += bs
        pbar.set_postfix(loss=run_loss/n, acc=run_acc/n)

    # ---- Validation
    model.eval()
    val_loss = 0.0; val_acc = 0.0; m = 0
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc=f"Epoch {epoch}/{EPOCHS} [val]"):
            x, y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x)
            loss = criterion(logits, y)
            bs = y.size(0)
            val_loss += loss.item() * bs
            val_acc  += (logits.argmax(1) == y).float().sum().item()
            m += bs
    val_loss /= max(1, m)
    val_acc  /= max(1, m)

    print(f"[E{epoch}] train_loss={run_loss/n:.4f} train_acc={run_acc/n:.4f} | val_loss={val_loss:.4f} val_acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), BEST_WEIGHTS)
        print(f"✅ Saved best to {BEST_WEIGHTS} (val_acc={best_val_acc:.4f})")

    scheduler.step()

print("Training complete. Best val_acc:", round(best_val_acc, 4))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 158MB/s]
/tmp/ipython-input-2023327034.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
Epoch 1/20 [train]:   0%|          | 0/354 [00:00<?, ?it/s]/tmp/ipython-input-2023327034.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
Epoch 1/20 [val]: 100%|██████████| 63/63 [00:08<00:00,  7.48it/s]


[E1] train_loss=0.8671 train_acc=0.9551 | val_loss=0.7079 val_acc=0.9982
✅ Saved best to model.pth (val_acc=0.9982)


Epoch 2/20 [val]: 100%|██████████| 63/63 [00:08<00:00,  7.21it/s]


[E2] train_loss=0.6995 train_acc=0.9995 | val_loss=0.6931 val_acc=1.0000
✅ Saved best to model.pth (val_acc=1.0000)


...

[E18] train_loss=0.6857 train_acc=1.0000 | val_loss=0.6858 val_acc=1.0000


Epoch 19/20 [val]: 100%|██████████| 63/63 [00:08<00:00,  7.13it/s]


[E19] train_loss=0.6856 train_acc=1.0000 | val_loss=0.6858 val_acc=1.0000


Epoch 20/20 [val]: 100%|██████████| 63/63 [00:13<00:00,  4.80it/s]

[E20] train_loss=0.6856 train_acc=1.0000 | val_loss=0.6858 val_acc=1.0000
Training complete. Best val_acc: 1.0


Train

In [5]:
EPOCHS = 12
LR = 3e-4
WEIGHT_DECAY = 1e-4
BEST_WEIGHTS = "model.pth"

model = build_model(num_classes=num_classes, pretrained=True).to(DEVICE)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

best_val_acc = 0.0

def accuracy(logits, y):
    return (logits.argmax(1) == y).float().mean().item()

for epoch in range(1, EPOCHS+1):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} [train]")
    run_loss = 0.0; run_acc = 0.0; n = 0

    for x, y in pbar:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            logits = model(x)
            loss = F.cross_entropy(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        bs = y.size(0)
        run_loss += loss.item() * bs
        run_acc  += (logits.argmax(1) == y).float().sum().item()
        n += bs
        pbar.set_postfix(loss=run_loss/n, acc=run_acc/n)

    # ---- Validation
    model.eval()
    val_loss = 0.0; val_acc = 0.0; m = 0
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc=f"Epoch {epoch}/{EPOCHS} [val]"):
            x, y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x)
            loss = F.cross_entropy(logits, y)
            bs = y.size(0)
            val_loss += loss.item() * bs
            val_acc  += (logits.argmax(1) == y).float().sum().item()
            m += bs
    val_loss /= max(1, m)
    val_acc  /= max(1, m)

    print(f"[E{epoch}] train_loss={run_loss/n:.4f} train_acc={run_acc/n:.4f} | val_loss={val_loss:.4f} val_acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), BEST_WEIGHTS)
        print(f"✅ Saved best to {BEST_WEIGHTS} (val_acc={best_val_acc:.4f})")

    scheduler.step()

print("Training complete. Best val_acc:", round(best_val_acc, 4))

/tmp/ipython-input-1078223694.py:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))
Epoch 1/12 [train]:   0%|          | 0/354 [00:00<?, ?it/s]/tmp/ipython-input-1078223694.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
Epoch 1/12 [val]: 100%|██████████| 63/63 [00:08<00:00,  7.62it/s]


[E1] train_loss=0.2006 train_acc=0.9516 | val_loss=0.0055 val_acc=1.0000
✅ Saved best to model.pth (val_acc=1.0000)


Epoch 2/12 [val]: 100%|██████████| 63/63 [00:08<00:00,  7.58it/s]


[E2] train_loss=0.0075 train_acc=0.9985 | val_loss=0.0575 val_acc=0.9847


Epoch 3/12 [val]: 100%|██████████| 63/63 [00:09<00:00,  6.82it/s]


...

[E10] train_loss=0.0001 train_acc=1.0000 | val_loss=0.0002 val_acc=1.0000


Epoch 11/12 [val]: 100%|██████████| 63/63 [00:09<00:00,  6.60it/s]


[E11] train_loss=0.0001 train_acc=1.0000 | val_loss=0.0002 val_acc=1.0000


Epoch 12/12 [val]: 100%|██████████| 63/63 [00:09<00:00,  6.76it/s]

[E12] train_loss=0.0001 train_acc=1.0000 | val_loss=0.0002 val_acc=1.0000
Training complete. Best val_acc: 1.0


Evaluate on official test split

In [6]:
import json
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

TEST_BATCH = 128

test_set = datasets.GTSRB(
    root=DATA_ROOT, split="test", download=True, transform=get_transforms(train=False)
)
test_loader = DataLoader(
    test_set, batch_size=TEST_BATCH, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)

IDX_TO_CLASS = load_label_map("label_map.json")
num_classes = len(IDX_TO_CLASS)

# Reload best model for test
eval_model = build_model(num_classes=num_classes, pretrained=False).to(DEVICE)
state = torch.load(BEST_WEIGHTS, map_location=DEVICE)
eval_model.load_state_dict(state)
eval_model.eval()

all_preds, all_labels = [], []
with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Testing"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = eval_model(x)
        preds = logits.argmax(1)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(y.cpu().numpy())

y_true = np.concatenate(all_labels)
y_pred = np.concatenate(all_preds)
acc = (y_true == y_pred).mean()
print(f"Test accuracy: {acc:.4f}")

report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
cm = confusion_matrix(y_true, y_pred)

with open("eval_report.json", "w") as f:
    json.dump(
        {"accuracy": float(acc), "per_class": report, "confusion_matrix": cm.tolist()},
        f, indent=2
    )

acc, report["macro avg"]["f1-score"]

100%|██████████| 89.0M/89.0M [00:00<00:00, 232MB/s]
100%|██████████| 99.6k/99.6k [00:00<00:00, 2.67MB/s]
Testing: 100%|██████████| 99/99 [00:29<00:00,  3.36it/s]

Test accuracy: 0.9814


(np.float64(0.9813935075217736), 0.9770393062079163)

Zip artifacts for download / Spaces

In [7]:
from zipfile import ZipFile

artifacts = ["model.pth", "label_map.json", "eval_report.json"]
for a in artifacts:
    print(a, "exists?" , Path(a).exists())

with ZipFile("traffic_sign_artifacts.zip", "w") as zf:
    for a in artifacts:
        if Path(a).exists():
            zf.write(a)
print("Wrote traffic_sign_artifacts.zip")

model.pth exists? True
label_map.json exists? True
eval_report.json exists? True
Wrote traffic_sign_artifacts.zip


In [11]:
# Pick 12 random TEST images and see top-1 accuracy quickly
import random, numpy as np, torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
from collections import Counter

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
test_set = datasets.GTSRB(root="./data", split="test", download=True, transform=None) # Remove transform here
IDX_TO_CLASS = load_label_map("label_map.json")

# reload best model
m = build_model(num_classes=len(IDX_TO_CLASS), pretrained=False).to(DEVICE)
m.load_state_dict(torch.load("model.pth", map_location=DEVICE)); m.eval()

def pred(img: Image.Image):
    # Apply transforms here after getting the PIL image
    x = get_transforms(train=False)(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad(): p = torch.softmax(m(x), -1)[0]
    i = int(p.argmax().item()); return i, float(p[i].item())

ok = 0
for _ in range(12):
    j = random.randrange(len(test_set))
    img, y = test_set[j] # Access image and target using dataset indexing
    i, conf = pred(img)
    ok += int(i==y)
print(f"Quick top-1 on 12 random test samples: {ok}/12 correct")

Quick top-1 on 12 random test samples: 12/12 correct
